In [ ]:
print("Test")

In [ ]:
import sys, os
from pathlib import Path

# Robustly find project root by looking for pyproject.toml or setup.py
def find_project_root(start_path: str | Path = '.') -> str:
    p = Path(start_path).resolve()
    for parent in [p] + list(p.parents):
        if (parent / 'pyproject.toml').exists() or (parent / 'setup.py').exists() or (parent / 'pyproject.lock').exists():
            return str(parent)
    # Fallback: go up two levels (original heuristic)
    return str(p.joinpath('..').joinpath('..').resolve())

project_root = find_project_root(os.getcwd())
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print('Project root added to path:', project_root)

In [ ]:
from research_and_analyst.utils.model_loader import ModelLoader
model_loader = ModelLoader()
model = model_loader.load_llm()

In [13]:
from typing import List
from typing_extensions import TypedDict
from pydantic import BaseModel, Field

In [ ]:
def create_analyst(state):
    pass

In [ ]:
def human_feedback(state):
    pass

In [ ]:
def should_continue(state):
    pass

In [ ]:
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langgraph.checkpoint.memory import MemorySaver

In [14]:
class Analyst(BaseModel):
    name: str = Field(description="Name of the analyst.")
    role: str = Field(description="Role of the analyst in the context of the topic.")
    affiliation: str = Field(description="Primary affiliation of the analyst.")
    description: str = Field(description="Description of the analyst focus, concerns, and motives.")
    
    @property
    def persona(self) -> str:
        return f"Name: {self.name}\nRole: {self.role}\nAffiliation: {self.affiliation}\nDescription: {self.description}\n"
    

In [16]:
Analyst(
    name="John Doe",
    role="UI / UX Designer",
    affiliation="AI Research LAB",
    description="I am UI/UX designer with a focus on creating intuitive and engaging user experiences for web and mobile applications."
    )

Analyst(name='John Doe', role='UI / UX Designer', affiliation='AI Research LAB', description='I am UI/UX designer with a focus on creating intuitive and engaging user experiences for web and mobile applications.')

In [17]:
analyst = Analyst(
    name="John Doe",
    role="UI / UX Designer",
    affiliation="AI Research LAB",
    description="I am UI/UX designer with a focus on creating intuitive and engaging user experiences for web and mobile applications."
    )

In [ ]:
analyst.name

'Name: John Doe\nRole: UI / UX Designer\nAffiliation: AI Research LAB\nDescription: I am UI/UX designer with a focus on creating intuitive and engaging user experiences for web and mobile applications.\n'

In [ ]:
builder = StateGraph()

In [ ]:
builder.add_node()
builder.add_node()


In [ ]:
builder.edge(START, create_analyst, HumanMessage(content="Hello, who won the world series in 2020?"))
builder.add_edge("create_analyst", "human_feedback", AIMessage(content="The Los Angeles Dodgers won the World Series in 2020."))
builder.conditional_edges("human_feedback", should_continue, ["create_analyst", END])

In [ ]:
memory = MemorySaver()

In [ ]:
graph = builder.compile(interrupt_before=["human_feedback"], checkpointer=memory)

In [ ]:
display(Image(graph.get_graph(xray=1).draw_mermaid_png()))